## CS431/631 Big Data Infrastructure
### Fall 2021 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Sai Anurag Neelisetty
* **ID:** _replace this with your UW student ID number_

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



Run the next block to download the text file (`Shakespeare.txt`) and the Python tokenizer module (`simple_tokenize.py`).

In [1]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [2]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
import itertools
unique_words=set()
unique_pairs=set()
# Now, let's tokenize Shakespeare's plays
with open('Shakespeare.txt') as f:
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)
        t=list(set(t))
        
        permutation_pairs=list(itertools.permutations(t,2))
        for x in permutation_pairs:
          unique_pairs.add(x)
        
        for word in t:
          unique_words.add(word)
        
print(len(unique_pairs))
print(len(unique_words))


# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

1969760
25975


In [6]:
list(unique_pairs)[0:7]

[('affliction', 'welcome'),
 ('crave', 'deeds'),
 ('to', 'isis'),
 ('heavier', 'to'),
 ('cool', 'you'),
 ('ides', 'tomorrow'),
 ('henceforth', 'letters')]

---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


In [61]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log
from collections import defaultdict, Counter
###################################################################################################################
#  replace this with your PMI analysis code, so that you can support the user interface below
#  it should read and tokenize Shakespeare.txt, and store enough information in Python data structures
#  to allow you to answer the PMI queries below

main_dict={}
token_dict={}
num_of_lines=0
def main_dict_check(wrd):
  tkn=main_dict.get(wrd)
  if tkn:
    return tkn
  else:
    tkn=defaultdict(int)
    main_dict[wrd]=tkn
    return tkn
  

def generate_dict():
  global num_of_lines
  with open('Shakespeare.txt') as f:
    for line in f:
      num_of_lines+=1
      t = set(simple_tokenize(line))
      for word in t:
        token_dict=main_dict_check(word)
        for x in t:
          token_dict[x]+=1
        main_dict[word]=token_dict

def PMI(cooccur_num,x,y):
  pmi_score=log((cooccur_num*num_of_lines)/(main_dict[x][x]*main_dict[y][y]),10)
  return pmi_score

def one_token_query(thd,tkn):
  occur_num=main_dict[tkn][tkn]
  dict_pairs_pmi=Counter()
  tkn_dict=main_dict[tkn]
  for k,v in tkn_dict.items():
    if v>thd:
      if k!=tkn:
        n,p=two_token_query([tkn,k])
        dict_pairs_pmi[(k,n)]=p
  return occur_num,dict_pairs_pmi

def two_token_query(query_tokens):
  co_occur_num=main_dict[query_tokens[0]][query_tokens[1]]
  pmi=PMI(co_occur_num,query_tokens[0],query_tokens[1])
  return co_occur_num,pmi
  

###################################################################################################################

###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################
generate_dict()
while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
        
        # Put code here to answer a One-Token Query with token q_tokens[0] and the specified threshold,
        # and output the result.
        xx,yy=one_token_query(threshold,q_tokens[0])
        # The print() statements below exist to show you the desired output format.
        # Replace them with your own output code, which should produce results in a similar format.
        print("  n({0}) = {1}".format(q_tokens[0],xx))
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))
        for (x,y),p in yy.most_common(5):
          print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0],x,y,p))       
        
          
        # print("    n({0},token2) = {1},  PMI({0},token2) = Y.YYY".format(q_tokens[0]))
        # print("    n({0},token3) = {1},  PMI({0},token3) = Y.YYY".format(q_tokens[0]))
        # print("    n({0},token4) = {1},  PMI({0},token4) = Y.YYY".format(q_tokens[0]))
        # print("    n({0},token5) = {1},  PMI({0},token5) = Y.YYY".format(q_tokens[0]))
        # in the above, all XXX values should be at least as large as the threshold

    elif len(q_tokens) == 2:
        # Put code here to answer a Two-Token Query with tokens q_tokens[0] and q_tokens[1]
        # As was the case for the One-Token query, the print statements below show the desired output format
        # Replace them with your own output code
        num,pmi=two_token_query(q_tokens)
        print("  n({0},{1}) = {2}".format(q_tokens[0],q_tokens[1],num))
        print("  PMI({0},{1}) = {2}".format(q_tokens[0],q_tokens[1],pmi))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


Input 1 or 2 space-separated tokens (return to quit): love hate
  n(love,hate) = 34
  PMI(love,hate) = 1.073565461516643
Input 1 or 2 space-separated tokens (return to quit): love
Input a positive integer frequency threshold: 12
  n(love) = 2020
  high PMI tokens with respect to love (threshold: 12):
    n(love,hate) = 34,  PMI(love,hate) = 1.073565461516643
    n(love,dear) = 35,  PMI(love,dear) = 0.6793208669926435
    n(love,valentine) = 16,  PMI(love,valentine) = 0.6307299182197897
    n(love,because) = 13,  PMI(love,because) = 0.6247375385273256
    n(love,rosalind) = 17,  PMI(love,rosalind) = 0.5981113661644437
Input 1 or 2 space-separated tokens (return to quit): 


---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:

*Replace this with your answer to Question 3.*


---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.